# BFGS Experimental Notebook

## Setup

### Imports

In [3]:
import numpy as np
from numpy import linalg as la
from scipy.optimize import line_search as off_shelf_line_search
from scipy.optimize import minimize
import pandas

from bfgs_funcs import Funcs
from bfgs import BFGS

from plotly.subplots import make_subplots
import plotly.graph_objects as go

### Data Generation

#### Load Data

In [4]:
# load the matrix
M1 = np.loadtxt('../data/M1.txt')
x0_m1 = np.loadtxt('../data/x0_m1.txt')
M2 = np.loadtxt('../data/M2.txt')
x0_m2 = np.loadtxt('../data/x0_m2.txt')
M3 = np.loadtxt('../data/M3.txt')
x0_m3 = np.loadtxt('../data/x0_m3.txt')
M4 = np.loadtxt('../data/M4.txt')
x0_m4 = np.loadtxt('../data/x0_m4.txt')
M5 = np.loadtxt('../data/M5.txt')
x0_m5 = np.loadtxt('../data/x0_m5.txt')

In [5]:
M1.shape, M2.shape, M3.shape, M4.shape, M5.shape

((1000, 1000), (10000, 100), (100, 1000), (100, 100), (1000, 100))

### Utils Functions

In [6]:
def error_report(matrix, fx, residuals, errors, max_iter = 1000, plot=False):
    computed_norm = np.sqrt(- fx)
    correct_norm = la.norm(matrix, 2)
    error = abs(correct_norm - computed_norm)
    
    num_iterations = len(residuals)
    if num_iterations != max_iter:
        print("Convergence reached in " + str(num_iterations) + " iterations")
    else:
        print("Convergence not reached")

    print("Computed Norm: " + str(computed_norm))
    print("True Norm: " + str(correct_norm))
    print("Error: " + str(error))
    
    if plot:
        fig = make_subplots(rows=1, cols=2, shared_xaxes=False)
        fig.add_trace( go.Scatter(x=list(range(0,len(errors))), y=errors,name="Errors"),
            row=1, col=1)
        fig.add_trace(
            go.Scatter(x=list(range(0,len(residuals))), y=residuals,name="Residuals"), row=1, col=2
        )
        fig.update_xaxes(title_text="Iterations", row=1, col=1)
        fig.update_yaxes(title_text="Error", row=1, col=1)
        fig.update_xaxes(title_text="Iterations", row=1, col=2)
        fig.update_yaxes(title_text="Residual", row=1, col=2)
        fig.update_layout(height=500, width=1000,title_text="Residual and Error plot").show()

In [7]:
def run_bfgs(matrix, c1=1e-4, c2=0.9, alg_method='O', ls_method='W', starting_point = [], max_iter=1000):
    funcs = Funcs(matrix)
    B0 = np.identity(matrix.shape[1])
    
    if len(starting_point) == 0:
        vector = np.round(np.random.randn(matrix.shape[1]),decimals = 3)
    else:
        vector = starting_point
    
    if ls_method == 'W':
        line_search_args = {'c1': c1, 'c2': c2}
    else:
        line_search_args = {'c1': c1}
        
    line_search_method = funcs.line_search_methods[ls_method]

    # Initialize the BFGS algorithm.
    bfgs = BFGS(funcs.func_, funcs.func_grad_, line_search_method, vector, B0, tol=1e-5, max_iter=max_iter, method=alg_method, line_search_args=line_search_args, verboose=False)

    # Run the algorithm.
    residuals, errors, result = bfgs.bfgs()

    # Evalutate Performance of the Single Run
    error_report(matrix, result, residuals, errors, max_iter=max_iter, plot=False)
    
    if len(starting_point) == 0:
        return vector, bfgs
    
    return bfgs

In [8]:
def off_shelf_opt(matrix, vector):
    funcs = Funcs(matrix)
    res = minimize(funcs.func_, vector, method='BFGS', jac=funcs.func_grad_, tol=1e-5, options={'gtol': 1e-6, 'disp': False})
    fx = res.fun
    
    num_iterations = res.nit
    if res.success:
        print("Convergence reached in " + str(num_iterations) + " iterations")
    else:
        print("Convergence not reached")
    
    computed_norm = np.sqrt(-fx)
    correct_norm = la.norm(matrix, 2)
    error = abs(computed_norm - correct_norm)
    
    print("Computed Norm: " + str(computed_norm))
    print("True Norm: " + str(correct_norm))
    print("Error: " + str(error))

## M1 Matrix

### Original BFGS

#### Base performance with loaded starting point (Wolfe-type line search)

In [9]:
w_o_m1_bfgs_a = run_bfgs(M1, alg_method='O', ls_method='W', c1=1e-4, c2=0.9, starting_point=x0_m1)

Convergence reached in 149 iterations
Computed Norm: 99.99999999989785
True Norm: 99.99999999999979
Error: 1.0193446087214397e-10


In [10]:
w_o_m1_bfgs_b = run_bfgs(M1, alg_method='O', ls_method='W', c1=0.1, c2=0.49, starting_point=x0_m1)

Convergence reached in 86 iterations
Computed Norm: 99.99999999996305
True Norm: 99.99999999999979
Error: 3.673505943879718e-11


#### Base performance with loaded starting point (Armijo-type line search)

In [11]:
a_o_m1_bfgs_a = run_bfgs(M1, alg_method='O', ls_method='A', c1=1e-4, starting_point=x0_m1)

Convergence reached in 37 iterations
Computed Norm: 99.99999999999976
True Norm: 99.99999999999979
Error: 2.842170943040401e-14


In [12]:
a_o_m1_bfgs_b = run_bfgs(M1, alg_method='O', ls_method='A', c1=0.1, starting_point=x0_m1)

/Users/JoeKifle/Desktop/CM_lastRun/CM-2Norm_matrix_problem/BFGS/bfgs.py:121: RuntimeWarning: divide by zero encountered in double_scalars
  ro = 1.0 / (np.dot(y, s))
/Users/JoeKifle/Desktop/CM_lastRun/CM-2Norm_matrix_problem/BFGS/bfgs.py:122: RuntimeWarning: invalid value encountered in multiply
  A1 = I - ro * s[:, np.newaxis] * y[np.newaxis, :]
/Users/JoeKifle/Desktop/CM_lastRun/CM-2Norm_matrix_problem/BFGS/bfgs.py:123: RuntimeWarning: invalid value encountered in multiply
  A2 = I - ro * y[:, np.newaxis] * s[np.newaxis, :]
/Users/JoeKifle/Desktop/CM_lastRun/CM-2Norm_matrix_problem/BFGS/bfgs.py:124: RuntimeWarning: invalid value encountered in multiply
  H_new = np.dot(A1, np.dot(H, A2)) + (ro * s[:, np.newaxis] * s[np.newaxis, :])


Convergence reached in 99 iterations
Computed Norm: nan
True Norm: 99.99999999999979
Error: nan


#### Average runtime over 10 runs (Wolfe-type line search)

In [13]:
%%timeit -r 10
# Measuring running time
w_o_m1_bfgs_a.bfgs()

34.4 s ± 5.76 s per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [14]:
%%timeit -r 10
# Measuring running time
w_o_m1_bfgs_b.bfgs()

22.3 s ± 2.41 s per loop (mean ± std. dev. of 10 runs, 1 loop each)


#### Average runtime over 10 runs (Armijo-type line search)

In [15]:
%%timeit -r 10
# Measuring running time
a_o_m1_bfgs_a.bfgs()

8.18 s ± 925 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [16]:
%%timeit -r 10
# Measuring running time
a_o_m1_bfgs_b.bfgs()

20.9 s ± 2.47 s per loop (mean ± std. dev. of 10 runs, 1 loop each)


#### Comparison with off-shelf BFGS minimizer

In [17]:
off_shelf_opt(M1, x0_m1)
funcs = Funcs(M1)

Convergence reached in 120 iterations
Computed Norm: 99.99999999998693
True Norm: 99.99999999999979
Error: 1.2860823517257813e-11


In [18]:
%%timeit -r 10
# Measuring running time
minimize(funcs.func_, x0_m1, method='BFGS', jac=funcs.func_grad_, options={'gtol': 1e-6, 'disp': False})

24.3 s ± 4.14 s per loop (mean ± std. dev. of 10 runs, 1 loop each)


#### Summary for Original BFGS

### Cautious BFGS

#### Base performance with loaded starting point (Wolfe-type line search)

In [19]:
w_c_m1_bfgs_a = run_bfgs(M1, alg_method='C', ls_method='W', c1=1e-4, c2=0.9, starting_point=x0_m1)

Convergence reached in 48 iterations
Computed Norm: 99.9999999999994
True Norm: 99.99999999999979
Error: 3.836930773104541e-13


In [20]:
w_c_m1_bfgs_b = run_bfgs(M1, alg_method='C', ls_method='W', c1=0.1, c2=0.49, starting_point=x0_m1)

Convergence reached in 47 iterations
Computed Norm: 99.99999999999937
True Norm: 99.99999999999979
Error: 4.121147867408581e-13


#### Base performance with loaded starting point (Armijo-type line search)

In [21]:
a_c_m1_bfgs_a = run_bfgs(M1, alg_method='C', ls_method='A', c1=1e-4, starting_point=x0_m1)

Convergence reached in 54 iterations
Computed Norm: 99.99999999999949
True Norm: 99.99999999999979
Error: 2.984279490192421e-13


In [22]:
a_c_m1_bfgs_b = run_bfgs(M1, alg_method='C', ls_method='A', c1=0.1, starting_point=x0_m1)

Convergence reached in 54 iterations
Computed Norm: 99.99999999999949
True Norm: 99.99999999999979
Error: 2.984279490192421e-13


#### Average runtime over 10 runs (Wolfe-type line search)

In [23]:
%%timeit -r 10
# Measuring running time
w_c_m1_bfgs_a.bfgs()

The slowest run took 4.03 times longer than the fastest. This could mean that an intermediate result is being cached.
13.5 s ± 6.5 s per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [24]:
%%timeit -r 10
# Measuring running time
w_c_m1_bfgs_b.bfgs()

16.1 s ± 4.59 s per loop (mean ± std. dev. of 10 runs, 1 loop each)


#### Average runtime over 10 runs (Armijo-type line search)

In [25]:
%%timeit -r 10
# Measuring running time
a_c_m1_bfgs_a.bfgs()

8.42 s ± 2.28 s per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [26]:
%%timeit -r 10
# Measuring running time
a_c_m1_bfgs_b.bfgs()

10.3 s ± 2.95 s per loop (mean ± std. dev. of 10 runs, 1 loop each)


#### Summary for Cautious BFGS

## M2 Matrix

### Original BFGS

#### Base performance with loaded starting point (Wolfe-type line search)

In [27]:
w_o_m2_bfgs_a = run_bfgs(M2, alg_method='O', ls_method='W', c1=1e-4, c2=0.9, starting_point=x0_m2)

Convergence reached in 445 iterations
Computed Norm: 109.46751097802968
True Norm: 109.46751097963192
Error: 1.6022454474295955e-09


In [28]:
w_o_m2_bfgs_b = run_bfgs(M2, alg_method='O', ls_method='W', c1=0.1, c2=0.49, starting_point=x0_m2)

Convergence reached in 169 iterations
Computed Norm: 109.46751097956667
True Norm: 109.46751097963192
Error: 6.52562448522076e-11


#### Base performance with loaded starting point (Armijo-type line search)

In [29]:
a_o_m2_bfgs_a = run_bfgs(M2, alg_method='O', ls_method='A', c1=1e-4, starting_point=x0_m2)

Convergence reached in 440 iterations
Computed Norm: 109.46751097793373
True Norm: 109.46751097963192
Error: 1.6981971384666394e-09


In [30]:
a_o_m2_bfgs_b = run_bfgs(M2, alg_method='O', ls_method='A', c1=0.1, starting_point=x0_m2)

Convergence reached in 241 iterations
Computed Norm: 109.46751097958969
True Norm: 109.46751097963192
Error: 4.2234660213580355e-11


#### Average runtime over 10 runs (Wolfe-type line search)

In [31]:
%%timeit -r 10
# Measuring running time
w_o_m2_bfgs_a.bfgs()

The slowest run took 14.90 times longer than the fastest. This could mean that an intermediate result is being cached.
1.78 s ± 1.73 s per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [32]:
%%timeit -r 10
# Measuring running time
w_o_m2_bfgs_b.bfgs()

623 ms ± 141 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


#### Average runtime over 10 runs (Armijo-type line search)

In [33]:
%%timeit -r 10
# Measuring running time
a_o_m2_bfgs_a.bfgs()

577 ms ± 279 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [34]:
%%timeit -r 10
# Measuring running time
a_o_m2_bfgs_b.bfgs()

The slowest run took 6.40 times longer than the fastest. This could mean that an intermediate result is being cached.
488 ms ± 307 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


#### Comparison with off-shelf BFGS minimizer

In [35]:
off_shelf_opt(M2, x0_m2)
funcs = Funcs(M2)

Convergence not reached
Computed Norm: 109.43610778289376
True Norm: 109.46751097963192
Error: 0.03140319673816805


In [36]:
%%timeit -r 10
# Measuring running time
minimize(funcs.func_, x0_m2, method='BFGS', jac=funcs.func_grad_, options={'gtol': 1e-6, 'disp': False})

The slowest run took 35.41 times longer than the fastest. This could mean that an intermediate result is being cached.
652 ms ± 806 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


#### Summary for Original BFGS

### Cautious BFGS

#### Base performance with loaded starting point (Wolfe-type line search)

In [37]:
w_c_m2_bfgs_a = run_bfgs(M2, alg_method='C', ls_method='W', c1=1e-4, c2=0.9, starting_point=x0_m2)

Convergence reached in 284 iterations
Computed Norm: 109.46751097952577
True Norm: 109.46751097963192
Error: 1.0615508472255897e-10


In [38]:
w_c_m2_bfgs_b = run_bfgs(M2, alg_method='C', ls_method='W', c1=0.1, c2=0.49, starting_point=x0_m2)

Convergence reached in 113 iterations
Computed Norm: 109.46751097962832
True Norm: 109.46751097963192
Error: 3.609557097661309e-12


#### Base performance with loaded starting point (Armijo-type line search)

In [39]:
a_c_m2_bfgs_a = run_bfgs(M2, alg_method='C', ls_method='A', c1=1e-4, starting_point=x0_m2)

Convergence reached in 787 iterations
Computed Norm: 109.46751097961493
True Norm: 109.46751097963192
Error: 1.6996182239381596e-11


In [40]:
a_c_m2_bfgs_b = run_bfgs(M2, alg_method='C', ls_method='A', c1=0.1, starting_point=x0_m2)

Convergence reached in 342 iterations
Computed Norm: 109.46751097963069
True Norm: 109.46751097963192
Error: 1.2363443602225743e-12


#### Average runtime over 10 runs (Wolfe-type line search)

In [41]:
%%timeit -r 10
# Measuring running time
w_c_m2_bfgs_a.bfgs()

258 ms ± 60.6 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [42]:
%%timeit -r 10
# Measuring running time
w_c_m2_bfgs_b.bfgs()

137 ms ± 43.2 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


#### Average runtime over 10 runs (Armijo-type line search)

In [43]:
%%timeit -r 10
# Measuring running time
a_c_m2_bfgs_a.bfgs()

The slowest run took 4.14 times longer than the fastest. This could mean that an intermediate result is being cached.
509 ms ± 271 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [44]:
%%timeit -r 10
# Measuring running time
a_c_m2_bfgs_b.bfgs()

The slowest run took 11.56 times longer than the fastest. This could mean that an intermediate result is being cached.
407 ms ± 398 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


#### Summary for Cautious BFGS

## M3 Matrix

### Original BFGS

#### Base performance with loaded starting point (Wolfe-type line search)

In [45]:
w_o_m3_bfgs_a = run_bfgs(M3, alg_method='O', ls_method='W', c1=1e-4, c2=0.9, starting_point=x0_m3)

Convergence reached in 265 iterations
Computed Norm: 41.065553272377436
True Norm: 41.06555327364635
Error: 1.2689156392298173e-09


In [46]:
w_o_m3_bfgs_b = run_bfgs(M3, alg_method='O', ls_method='W', c1=0.1, c2=0.49, starting_point=x0_m3)

Convergence reached in 172 iterations
Computed Norm: 41.06555327109088
True Norm: 41.06555327364635
Error: 2.555474054588558e-09


#### Base performance with loaded starting point (Armijo-type line search)

In [47]:
a_o_m3_bfgs_a = run_bfgs(M3, alg_method='O', ls_method='A', c1=1e-4, starting_point=x0_m3)

Convergence reached in 66 iterations
Computed Norm: 39.31254846263882
True Norm: 41.06555327364635
Error: 1.7530048110075285


In [48]:
a_o_m3_bfgs_b = run_bfgs(M3, alg_method='O', ls_method='A', c1=0.1, starting_point=x0_m3)

Convergence reached in 223 iterations
Computed Norm: 41.06555327338661
True Norm: 41.06555327364635
Error: 2.597388970571046e-10


#### Average runtime over 10 runs (Wolfe-type line search)

In [49]:
%%timeit -r 10
# Measuring running time
w_o_m3_bfgs_a.bfgs()

1min 14s ± 24.6 s per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [50]:
%%timeit -r 10
# Measuring running time
w_o_m3_bfgs_b.bfgs()

59.4 s ± 11.8 s per loop (mean ± std. dev. of 10 runs, 1 loop each)


#### Average runtime over 10 runs (Armijo-type line search)

In [51]:
%%timeit -r 10
# Measuring running time
a_o_m3_bfgs_a.bfgs()

KeyboardInterrupt: 

In [ ]:
%%timeit -r 10
# Measuring running time
a_o_m3_bfgs_b.bfgs()

31.5 s ± 629 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


#### Comparison with off-shelf BFGS minimizer

In [ ]:
off_shelf_opt(M3, x0_m3)
funcs = Funcs(M3)

Convergence reached in 294 iterations
Computed Norm: 41.065553271820015
True Norm: 41.06555327364635
Error: 1.826336415433616e-09


In [ ]:
%%timeit -r 10
# Measuring running time
minimize(funcs.func_, x0_m3, method='BFGS', jac=funcs.func_grad_, options={'gtol': 1e-6, 'disp': False})

34.7 s ± 818 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


#### Summary for Original BFGS

### Cautious BFGS

#### Base performance with loaded starting point (Wolfe-type line search)

In [ ]:
w_c_m3_bfgs_a = run_bfgs(M3, alg_method='C', ls_method='W', c1=1e-4, c2=0.9, starting_point=x0_m3)

Convergence reached in 100 iterations
Computed Norm: 41.065553273638365
True Norm: 41.06555327364635
Error: 7.986500349943526e-12


In [ ]:
w_c_m3_bfgs_b = run_bfgs(M3, alg_method='C', ls_method='W', c1=0.1, c2=0.49, starting_point=x0_m3)

Convergence reached in 75 iterations
Computed Norm: 41.065553273632354
True Norm: 41.06555327364635
Error: 1.3997691894473974e-11


#### Base performance with loaded starting point (Armijo-type line search)

In [ ]:
a_c_m3_bfgs_a = run_bfgs(M3, alg_method='C', ls_method='A', c1=1e-4, starting_point=x0_m3)

Convergence reached in 117 iterations
Computed Norm: 41.065553273643665
True Norm: 41.06555327364635
Error: 2.6858515411731787e-12


In [ ]:
a_c_m3_bfgs_b = run_bfgs(M3, alg_method='C', ls_method='A', c1=0.1, starting_point=x0_m3)

Convergence reached in 117 iterations
Computed Norm: 41.065553273643665
True Norm: 41.06555327364635
Error: 2.6858515411731787e-12


#### Average runtime over 10 runs (Wolfe-type line search)

In [ ]:
%%timeit -r 10
# Measuring running time
w_c_m3_bfgs_a.bfgs()

8.51 s ± 430 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [ ]:
%%timeit -r 10
# Measuring running time
w_c_m3_bfgs_b.bfgs()

11.6 s ± 1.73 s per loop (mean ± std. dev. of 10 runs, 1 loop each)


#### Average runtime over 10 runs (Armijo-type line search)

In [ ]:
%%timeit -r 10
# Measuring running time
a_c_m3_bfgs_a.bfgs()

6.37 s ± 77.8 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [ ]:
%%timeit -r 10
# Measuring running time
a_c_m3_bfgs_b.bfgs()

6.37 s ± 91.4 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


#### Summary for Cautious BFGS

## M4 Matrix

### Original BFGS

#### Base performance with loaded starting point (Wolfe-type line search)

In [ ]:
w_o_m4_bfgs_a = run_bfgs(M4, alg_method='O', ls_method='W', c1=1e-4, c2=0.9, starting_point=x0_m4)

Convergence reached in 192 iterations
Computed Norm: 19.356959367506004
True Norm: 19.356959368936618
Error: 1.4306138496067433e-09


In [ ]:
w_o_m4_bfgs_b = run_bfgs(M4, alg_method='O', ls_method='W', c1=0.1, c2=0.49, starting_point=x0_m4)

Convergence reached in 138 iterations
Computed Norm: 19.3569593673212
True Norm: 19.356959368936618
Error: 1.6154189097505878e-09


#### Base performance with loaded starting point (Armijo-type line search)

In [ ]:
a_o_m4_bfgs_a = run_bfgs(M4, alg_method='O', ls_method='A', c1=1e-4, starting_point=x0_m4)

Convergence reached in 211 iterations
Computed Norm: 19.356959366896895
True Norm: 19.356959368936618
Error: 2.0397230571234104e-09


In [ ]:
a_o_m4_bfgs_b = run_bfgs(M4, alg_method='O', ls_method='A', c1=0.1, starting_point=x0_m4)

Convergence reached in 211 iterations
Computed Norm: 19.356959366896895
True Norm: 19.356959368936618
Error: 2.0397230571234104e-09


#### Average runtime over 10 runs (Wolfe-type line search)

In [ ]:
%%timeit -r 10
# Measuring running time
w_o_m4_bfgs_a.bfgs()

161 ms ± 17.5 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [ ]:
%%timeit -r 10
# Measuring running time
w_o_m4_bfgs_b.bfgs()

139 ms ± 52.6 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


#### Average runtime over 10 runs (Armijo-type line search)

In [ ]:
%%timeit -r 10
# Measuring running time
a_o_m4_bfgs_a.bfgs()

160 ms ± 26.2 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [ ]:
%%timeit -r 10
# Measuring running time
a_o_m4_bfgs_b.bfgs()

187 ms ± 51.3 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


#### Comparison with off-shelf BFGS minimizer

In [ ]:
off_shelf_opt(M4, x0_m4)
funcs = Funcs(M4)

Convergence reached in 55 iterations
Computed Norm: 18.42502730725014
True Norm: 19.356959368936618
Error: 0.9319320616864779


In [ ]:
%%timeit -r 10
# Measuring running time
minimize(funcs.func_, x0_m4, method='BFGS', jac=funcs.func_grad_, options={'gtol': 1e-6, 'disp': False})

43.2 ms ± 7.09 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


#### Summary for Original BFGS

### Cautious BFGS

#### Base performance with loaded starting point (Wolfe-type line search)

In [ ]:
w_c_m4_bfgs_a = run_bfgs(M4, alg_method='C', ls_method='W', c1=1e-4, c2=0.9, starting_point=x0_m4)

Convergence reached in 58 iterations
Computed Norm: 19.35695936892961
True Norm: 19.356959368936618
Error: 7.009504088273388e-12


In [ ]:
w_c_m4_bfgs_b = run_bfgs(M4, alg_method='C', ls_method='W', c1=0.1, c2=0.49, starting_point=x0_m4)

Convergence reached in 50 iterations
Computed Norm: 19.356959368930855
True Norm: 19.356959368936618
Error: 5.7625015870144125e-12


#### Base performance with loaded starting point (Armijo-type line search)

In [ ]:
a_c_m4_bfgs_a = run_bfgs(M4, alg_method='C', ls_method='A', c1=1e-4, starting_point=x0_m4)

Convergence reached in 53 iterations
Computed Norm: 19.356959368933875
True Norm: 19.356959368936618
Error: 2.7426949600339867e-12


In [ ]:
a_c_m4_bfgs_b = run_bfgs(M4, alg_method='C', ls_method='A', c1=0.1, starting_point=x0_m4)

Convergence reached in 53 iterations
Computed Norm: 19.356959368933875
True Norm: 19.356959368936618
Error: 2.7426949600339867e-12


#### Average runtime over 10 runs (Wolfe-type line search)

In [ ]:
%%timeit -r 10
# Measuring running time
w_c_m4_bfgs_a.bfgs()

41.2 ms ± 3.23 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [ ]:
%%timeit -r 10
# Measuring running time
w_c_m4_bfgs_b.bfgs()

38.6 ms ± 2.74 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


#### Average runtime over 10 runs (Armijo-type line search)

In [ ]:
%%timeit -r 10
# Measuring running time
a_c_m4_bfgs_a.bfgs()

28.5 ms ± 2.58 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [ ]:
%%timeit -r 10
# Measuring running time
a_c_m4_bfgs_b.bfgs()

28.4 ms ± 3.2 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


#### Summary for Cautious BFGS

## M5 Matrix

### Original BFGS

#### Base performance with loaded starting point (Wolfe-type line search)

In [ ]:
w_o_m5_bfgs_a = run_bfgs(M5, alg_method='O', ls_method='W', c1=1e-4, c2=0.9, starting_point=x0_m5)

Convergence reached in 66 iterations
Computed Norm: 23.624892381956926
True Norm: 23.62489238197046
Error: 1.3535839116229909e-11


In [ ]:
w_o_m5_bfgs_b = run_bfgs(M5, alg_method='O', ls_method='W', c1=0.1, c2=0.49, starting_point=x0_m5)

Convergence reached in 52 iterations
Computed Norm: 23.624892381944633
True Norm: 23.62489238197046
Error: 2.5828228444879642e-11


#### Base performance with loaded starting point (Armijo-type line search)

In [ ]:
a_o_m5_bfgs_a = run_bfgs(M5, alg_method='O', ls_method='A', c1=1e-4, starting_point=x0_m5)

Convergence reached in 66 iterations
Computed Norm: 23.624892381956926
True Norm: 23.62489238197046
Error: 1.3535839116229909e-11


In [ ]:
a_o_m5_bfgs_b = run_bfgs(M5, alg_method='O', ls_method='A', c1=0.1, starting_point=x0_m5)

Convergence reached in 66 iterations
Computed Norm: 23.624892381956926
True Norm: 23.62489238197046
Error: 1.3535839116229909e-11


#### Average runtime over 10 runs (Wolfe-type line search)

In [ ]:
%%timeit -r 10
# Measuring running time
w_o_m5_bfgs_a.bfgs()

50.3 ms ± 3.04 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [ ]:
%%timeit -r 10
# Measuring running time
w_o_m5_bfgs_b.bfgs()

43.3 ms ± 7.49 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


#### Average runtime over 10 runs (Armijo-type line search)

In [ ]:
%%timeit -r 10
# Measuring running time
a_o_m5_bfgs_a.bfgs()

44.7 ms ± 4.63 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [ ]:
%%timeit -r 10
# Measuring running time
a_o_m5_bfgs_b.bfgs()

97.4 ms ± 41.4 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


#### Comparison with off-shelf BFGS minimizer

In [ ]:
off_shelf_opt(M5, x0_m5)
funcs = Funcs(M5)

Convergence reached in 57 iterations
Computed Norm: 23.624892381968756
True Norm: 23.62489238197046
Error: 1.7053025658242404e-12


In [ ]:
%%timeit -r 10
# Measuring running time
minimize(funcs.func_, x0_m5, method='BFGS', jac=funcs.func_grad_, options={'gtol': 1e-6, 'disp': False})

The slowest run took 13.64 times longer than the fastest. This could mean that an intermediate result is being cached.
105 ms ± 120 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


#### Summary for Original BFGS

### Cautious BFGS

#### Base performance with loaded starting point (Wolfe-type line search)

In [ ]:
w_c_m5_bfgs_a = run_bfgs(M5, alg_method='C', ls_method='W', c1=1e-4, c2=0.9, starting_point=x0_m5)

Convergence reached in 49 iterations
Computed Norm: 23.62489238196187
True Norm: 23.62489238197046
Error: 8.590461675339611e-12


In [ ]:
w_c_m5_bfgs_b = run_bfgs(M5, alg_method='C', ls_method='W', c1=0.1, c2=0.49, starting_point=x0_m5)

Convergence reached in 40 iterations
Computed Norm: 23.624892381959825
True Norm: 23.62489238197046
Error: 1.06368247543287e-11


#### Base performance with loaded starting point (Armijo-type line search)

In [ ]:
a_c_m5_bfgs_a = run_bfgs(M5, alg_method='C', ls_method='A', c1=1e-4, starting_point=x0_m5)

Convergence reached in 55 iterations
Computed Norm: 23.624892381965147
True Norm: 23.62489238197046
Error: 5.314859663485549e-12


In [ ]:
a_c_m5_bfgs_b = run_bfgs(M5, alg_method='C', ls_method='A', c1=0.1, starting_point=x0_m5)

Convergence reached in 55 iterations
Computed Norm: 23.624892381965147
True Norm: 23.62489238197046
Error: 5.314859663485549e-12


#### Average runtime over 10 runs (Wolfe-type line search)

In [ ]:
%%timeit -r 10
# Measuring running time
w_c_m5_bfgs_a.bfgs()

The slowest run took 6.27 times longer than the fastest. This could mean that an intermediate result is being cached.
104 ms ± 49.3 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [ ]:
%%timeit -r 10
# Measuring running time
w_c_m5_bfgs_b.bfgs()

The slowest run took 4.43 times longer than the fastest. This could mean that an intermediate result is being cached.
61.7 ms ± 36.9 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


#### Average runtime over 10 runs (Armijo-type line search)

In [ ]:
%%timeit -r 10
# Measuring running time
a_c_m5_bfgs_a.bfgs()

The slowest run took 8.24 times longer than the fastest. This could mean that an intermediate result is being cached.
117 ms ± 44.6 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [ ]:
%%timeit -r 10
# Measuring running time
a_c_m5_bfgs_b.bfgs()

The slowest run took 4.34 times longer than the fastest. This could mean that an intermediate result is being cached.
67.9 ms ± 41.5 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


#### Summary for Cautious BFGS

## Summary for Original BFGS

## Summary for Cautious BFGS